## Setup

Print path of current directory:

In [ ]:
;pwd

Check that `Apophis.jl` environment is active:

In [ ]:
]st

In [ ]:
using Revise

In [ ]:
using PlanetaryEphemeris

In [ ]:
using Apophis

In [ ]:
using JLD, TaylorSeries

Setup `TaylorN` variables:

In [ ]:
#dq = set_variables("ξ", order=5, numvars=6)
#dq = set_variables("ξ", order=6, numvars=7)
#dq = set_variables("ξ", order=7, numvars=7)
dq = set_variables("ξ", order=5, numvars=7)

In [ ]:
using Dates, LinearAlgebra, DelimitedFiles, Statistics

In [ ]:
using JuliaDB

In [ ]:
Apophis.loadjpleph();

## Process radar data

Process radar (time-delay/Doppler) data:

In [ ]:
apophis_data = process_radar_data_jpl("../data/Apophis_JPL_data.dat");

In [ ]:
# compute time-delay, Doppler-shift values from JPL ephemerides
tdelay_JPL_v1, dshift_JPL_v1 = delay_doppler(apophis_data);

In [ ]:
delay_index = findall(x->x.delay_units=="us", apophis_data)
doppler_index = findall(x->x.doppler_units=="Hz", apophis_data);

In [ ]:
tdelay_jpl_obs = [x.delay for x in apophis_data][delay_index]
dshift_jpl_obs = [x.doppler for x in apophis_data][doppler_index]

tdelay_jpl_obs_sigma = [x.delay_sigma for x in apophis_data][delay_index]
dshift_jpl_obs_sigma = [x.doppler_sigma for x in apophis_data][doppler_index];

In [ ]:
dshift_jpl_freq = [x.freq for x in apophis_data][doppler_index]; #MHz !!!!!!!!!

In [ ]:
tdelay_jpl_obs_jd_utc = [datetime2julian(x.utcepoch) for x in apophis_data][delay_index]
dshift_jpl_obs_jd_utc = [datetime2julian(x.utcepoch) for x in apophis_data][doppler_index];

In [ ]:
transmitter_freq_MHz = [x.freq for x in apophis_data];

## Process computed delay and Dopplers

In [ ]:
#radec_bwd_jld = "radec_tho13_o5v7.jld"
#radec_fwd_jld = "radec_vokr15_o5v7.jld"
#deldop_bwd_jld = "deldop_Apophis_JPL_data_2005_2006_25ASTF13.jld"
#deldop_fwd_jld = "deldop_Apophis_JPL_data_2012_2013_25ASTF13.jld"
deldop_bwd_jld = "deldop_m5y_o5v7_x0_s197_BIG16AST.jld"
deldop_fwd_jld = "deldop_p6y_o5v7_x0_s197_BIG16AST.jld"

#vra_bwd  = JLD.load(radec_bwd_jld, "vra")
#vdec_bwd = JLD.load(radec_bwd_jld, "vdec")
#vra_fwd  = JLD.load(radec_fwd_jld, "vra")
#vdec_fwd = JLD.load(radec_fwd_jld, "vdec")
#vra_bwd = select(tholen13_data, :α_comp)
#vdec_bwd = select(tholen13_data, :δ_comp)
#vra_fwd = select(vokr15_data, :α_comp)
#vdec_fwd = select(vokr15_data, :δ_comp)

vdel_bwd = JLD.load(deldop_bwd_jld, "vdel")
vdop_bwd = JLD.load(deldop_bwd_jld, "vdop")
vdel_fwd = JLD.load(deldop_fwd_jld, "vdel")
vdop_fwd = JLD.load(deldop_fwd_jld, "vdop")

vdel = vcat(vdel_bwd, vdel_fwd)
vdop = vcat(vdop_bwd, vdop_fwd)
#vra = vcat(vra_bwd, vra_fwd)
#vdec = vcat(vdec_bwd, vdec_fwd);

In [ ]:
vdel_bwd[1].order

In [ ]:
# get number of TaylorN variables
nv = get_numvars()

In [ ]:
A2_JPL = -5.592839897872

In [ ]:
xyz_199 = [-0.96330181481543, 0.5100291398744977, 0.16528030015848072]
uvw_199 = [-0.007118874645519018, -0.0120612341605072, -0.004669513801491289]

xyz_197 = [-0.9633018953468989, 0.5100289806011301, 0.16528024397505386]
uvw_197 = [-0.0071188720570829036, -0.012061235689040535, -0.0046695143453363164];

In [ ]:
(xyz_197-xyz_199)*au

In [ ]:
(uvw_197-uvw_199)*au/daysec

In [ ]:
# JPL#199 nominal initial condition (wrt to our initial nominal solution)
x0_JPL = vcat(1e8vcat(xyz_199-xyz_197,uvw_199-uvw_197), A2_JPL)

In [ ]:
# https://ssd.jpl.nasa.gov/sbdb.cgi?sstr=99942;orb=0;cov=1;log=0;cad=0#elem
Γ_JPL199 = [
 2.858062040774483E-17  -2.531903029559859E-17  2.451271598753632E-14  5.341143529000106E-14 -5.064059304794175E-14 -2.316088496620918E-16 -9.159706240530915E-23
-2.531903029559859E-17   2.250437378457998E-17 -2.122266051203493E-14 -5.100944859184798E-14  4.826834522209508E-14  2.76636664162405E-16   7.744081722922498E-23
 2.451271598753632E-14  -2.122266051203493E-14  2.501437350446983E-11  1.16213242311946E-11  -1.057653022558682E-11  4.597741246907787E-13 -1.026132427113056E-19
 5.341143529000106E-14  -5.100944859184798E-14  1.16213242311946E-11   4.437388022822618E-10 -4.338840441053813E-10 -6.817498150073878E-12 -4.324000765783613E-21
-5.064059304794175E-14   4.826834522209508E-14 -1.057653022558682E-11 -4.338840441053813E-10  4.261149273549589E-10  6.697032883689781E-12  1.139615135464918E-20
-2.316088496620918E-16   2.76636664162405E-16   4.597741246907787E-13 -6.817498150073878E-12  6.697032883689781E-12  1.22672053287042E-13  -2.524941685627351E-21
-9.159706240530915E-23   7.744081722922498E-23 -1.026132427113056E-19 -4.324000765783613E-21  1.139615135464918E-20 -2.524941685627351E-21  4.846398125111792E-28
]

In [ ]:
sqrt(1/(inv(Γ_JPL199)[7,7]))

## Process optical data

Following Vokrouhlicky et al. (2015), we process optical astrometric data (right ascension / declination) from Tholen et al. (2013), MPEC 2014-R71 (Mauna Kea), Pan-STARRS PS1, and Magdalena Ridge Observatory:

In [ ]:
# recover data from saved .jdb file
#_tholen13_data = JuliaDB.load("radec_tholen13_mpc_formatted_25ASTF13.jdb")
#_vokr15_data = JuliaDB.load("radec_vokr15_mpc_formatted_25ASTF13.jdb")
_tholen13_data = JuliaDB.load("radec_tholen13_mpc_formatted_x0_s197_BIG16AST.jdb")
_vokr15_data = JuliaDB.load("radec_vokr15_mpc_formatted_x0_s197_BIG16AST.jdb")

# compute debiasing corrections from Farnocchia et al (2015) table
tho13_sjpl_t = radec_table("../data/tholen13_mpc_formatted.dat", debias_table="2014")
vok15_sjpl_t = radec_table("../data/vokr15_mpc_formatted.dat", debias_table="2014")
ras_corr_tho13 = select(tho13_sjpl_t, :α_corr)
dec_corr_tho13 = select(tho13_sjpl_t, :δ_corr)
ras_corr_vok15 = select(vok15_sjpl_t, :α_corr)
dec_corr_vok15 = select(vok15_sjpl_t, :δ_corr);

In [ ]:
# construct tables including "2014" debiasing corrections 

tholen13_data = insertcolsafter(
    _tholen13_data, 
    length(columns(_tholen13_data)), 
    :α_corr_2014 => ras_corr_tho13, 
    :δ_corr_2014 => dec_corr_tho13
)

vokr15_data = insertcolsafter(
    _vokr15_data, 
    length(columns(_vokr15_data)), 
    :α_corr_2014 => ras_corr_vok15, 
    :δ_corr_2014 => dec_corr_vok15
);

In [ ]:
# used in plots vs time
tv_opt = select(tholen13_data, :dt_utc_obs => x->datetime2julian(x))
tv_opt_v15 = select(vokr15_data, :dt_utc_obs => x->datetime2julian(x));

In [ ]:
mean(select(tholen13_data, (:α_obs, :α_comp)=>x->x[1]-x[2]()))

In [ ]:
mean(select(tholen13_data, (:δ_obs, :δ_comp)=>x->x[1]-x[2]()))

In [ ]:
mean(select(vokr15_data, (:α_obs, :α_comp)=>x->x[1]-x[2]()))

In [ ]:
mean(select(vokr15_data, (:δ_obs, :δ_comp)=>x->x[1]-x[2]()))

In [ ]:
dates_t13 = select(tholen13_data, :dt_utc_obs => x->Date(x))
dates_v15 = select(vokr15_data, :dt_utc_obs => x->Date(x))

# vector of number of observations per night, Farnocchia et al (2013)
Nv_t13 = [count(x->x==i, dates_t13) for i in dates_t13]
# vector of number of observations per night, Vokrouhlicky et al (2015)
Nv_v15 = [count(x->x==i, dates_v15) for i in dates_v15];

In [ ]:
#weight scale
w8_scale(i) = i>5 ? i/5 : 1.0 # Farnocchia et al 2013

In [ ]:
relax_t13 = [w8_scale(i) for i in Nv_t13];

In [ ]:
relax_v15 = [w8_scale(i) for i in Nv_v15]
;

In [ ]:
# error component that accounts for intrinsic bias of 2MASS catalog at J2000 epoch
σ_B = 0.04 # arcseconds 

# time dependent uncertainty component related to proper motion biases:
σ_PM = select(tholen13_data, :yr=>x->0.05*(x-2000)/(2011-2000)) # arcseconds
;

In [ ]:
tho13_errors = readdlm("../data/tholenetal2013_opterror.dat", ',');

# astrometric solution (fit) uncertainty
Δα_a = Float64.(tho13_errors[:,1]) # arcseconds
Δδ_a = Float64.(tho13_errors[:,2]) # arcseconds

# object centroiding uncertainty
Δα_χ = Float64.(tho13_errors[:,3]) # arcseconds
Δδ_χ = Float64.(tho13_errors[:,4]) # arcseconds

# tracking uncertainty
Δα_t = Float64.(tho13_errors[:,5]) # arcseconds
Δδ_t = Float64.(tho13_errors[:,6]) # arcseconds

#Construct α, δ weights according to Farnocchia et al. (2013) weighting scheme:
#w_α_f13 = @__dot__ (1/(Δα_a^2 + Δα_χ^2 + Δα_t^2 + Nv_t13*σ_B^2 + Nv_t13*σ_PM^2 ))/relax_t13 # Eq. (2), Farnocchia et al. (2013)
#w_δ_f13 = @__dot__ (1/(Δδ_a^2 + Δδ_χ^2 + Δδ_t^2 + Nv_t13*σ_B^2 + Nv_t13*σ_PM^2 ))/relax_t13; # Eq. (3), Farnocchia et al. (2013)

w_α_f13 = @__dot__ (1/(Δα_a^2 + Δα_χ^2 + Δα_t^2))/relax_t13 # Eq. (2), Farnocchia et al. (2013)
w_δ_f13 = @__dot__ (1/(Δδ_a^2 + Δδ_χ^2 + Δδ_t^2))/relax_t13; # Eq. (3), Farnocchia et al. (2013)

In [ ]:
# time dependent uncertainty component related to proper motion biases:
σ_PM_v15 = select(vokr15_data, :yr=>x->0.05*(x-2000)/(2011-2000)) # arcseconds;

In [ ]:
# Magdalena Ridge weights
w_α_mro = select(filter(x->x.obscode=="H01", vokr15_data), :obscode => x->1/(0.30)^2) # 0.15,0.30
w_δ_mro = w_α_mro

#Mauna Kea (MPEC 2014-R71) weights
w_α_mk = select(filter(x->x.obscode=="568", vokr15_data), :obscode => x->1/(0.13)^2)
w_δ_mk = w_α_mk

#PanSTARRS-PS1 weights
ind_PS1 = select(vokr15_data, :obscode => x->x=="F51")
#w_α_ps1 = @__dot__ 1/((0.30)^2 + Nv_v15[ind_PS1]*σ_B^2 + Nv_v15[ind_PS1]*σ_PM_v15[ind_PS1]^2 )
w_α_ps1 = select(filter(x->x.obscode=="F51", vokr15_data), :obscode => x->1/(0.15)^2) # was 0.15,0.30
w_δ_ps1 = w_α_ps1

# Vokrouhlicky et al (2015) weights
w_α_v15 = vcat(w_α_mro, w_α_mk, w_α_ps1)./relax_v15
w_δ_v15 = vcat(w_δ_mro, w_δ_mk, w_δ_ps1)./relax_v15;

## Compute optical measurements residuals and weights

In [ ]:
radec_data_ALL = merge(tholen13_data, vokr15_data);

In [ ]:
radec_data_noMK = filter(x->!(x.yr>2008 && x.obscode=="568"), radec_data_ALL, select=(:yr,:obscode));

In [ ]:
no_mk_ind_t = select(radec_data_ALL, (:yr,:obscode)=>x->!(x.yr>2008 && x.obscode=="568"));

In [ ]:
# observed

α_obs_ALL = select(radec_data_ALL, :α_obs) #arcseconds (includes cos(dec))
δ_obs_ALL = select(radec_data_ALL, :δ_obs) #arcseconds

# computed

α_comp_ALL = select(radec_data_ALL, :α_comp) #arcseconds (includes cos(dec))
δ_comp_ALL = select(radec_data_ALL, :δ_comp) #arcseconds

# debiasing

α_corr_2014 = select(radec_data_ALL, :α_corr_2014)
δ_corr_2014 = select(radec_data_ALL, :δ_corr_2014)
α_corr_2018 = select(radec_data_ALL, :α_corr)
δ_corr_2018 = select(radec_data_ALL, :δ_corr)

# weights

W_ra = vcat(w_α_f13, w_α_v15)
W_dec = vcat(w_δ_f13, w_δ_v15)
W_ra_noMK = W_ra[no_mk_ind_t]
W_dec_noMK = W_dec[no_mk_ind_t]
W_radec = vcat(W_ra, W_dec)
W_radec_noMK = vcat(W_ra_noMK, W_dec_noMK)

# residuals

res_ras_RAW = α_obs_ALL .- α_comp_ALL # right ascension residuals, arcseconds
res_dec_RAW = δ_obs_ALL .- δ_comp_ALL # declination residuals, arcseconds

res_ras_2014 = (α_obs_ALL .- α_corr_2014) .- α_comp_ALL # right ascension residuals, arcseconds
res_dec_2014 = (δ_obs_ALL .- δ_corr_2014) .- δ_comp_ALL # declination residuals, arcseconds

res_ras_2018 = (α_obs_ALL .- α_corr_2018) .- α_comp_ALL # right ascension residuals, arcseconds
res_dec_2018 = (δ_obs_ALL .- δ_corr_2018) .- δ_comp_ALL # declination residuals, arcseconds

res_ras_RAW_noMK = res_ras_RAW[no_mk_ind_t]
res_dec_RAW_noMK = res_dec_RAW[no_mk_ind_t]

res_ras_2014_noMK = res_ras_2014[no_mk_ind_t]
res_dec_2014_noMK = res_dec_2014[no_mk_ind_t]

res_ras_2018_noMK = res_ras_2018[no_mk_ind_t]
res_dec_2018_noMK = res_dec_2018[no_mk_ind_t]

### residuals and weights with/without Mauna-Kea selection, 2014/2018 debiasing

res_radec_RAW = vcat(res_ras_RAW, res_dec_RAW) # α, δ residuals
W_radec_RAW = W_radec; # Farnocchia et al (2013) weighting scheme

res_radec_RAW_noMK = vcat(res_ras_RAW_noMK, res_dec_RAW_noMK) # α, δ residuals
W_radec_RAW_noMK = W_radec_noMK; # Farnocchia et al (2013) weighting scheme

#RA/DEC observations residuals and weights, using Farnocchia et al. (2015) debiasing table, **WITHOUT** Mauna Kea (MPEC 2014-R71) obs
res_radec_2014 = vcat(res_ras_2014, res_dec_2014) # α, δ residuals
W_radec_2014 = W_radec; # Farnocchia et al (2013) weighting scheme

###RA/DEC observations residuals and weights, using Farnocchia et al. (2015) debiasing table, **WITHOUT** Mauna Kea (MPEC 2014-R71) obs
res_radec_2014_noMK = vcat(res_ras_2014_noMK, res_dec_2014_noMK) # α, δ residuals
W_radec_2014_noMK = W_radec_noMK; # Farnocchia et al (2013) weighting scheme

#RA/DEC observations residuals and weights, using Eggl et al. (2020) debiasing table, **WITH** Mauna Kea (MPEC 2014-R71) obs
res_radec_2018 = vcat(res_ras_2018, res_dec_2018) # α, δ residuals
W_radec_2018 = W_radec # Farnocchia et al (2013) weighting scheme

#RA/DEC observations residuals and weights, using Eggl et al. (2020) debiasing table, **WITHOUT** Mauna Kea (MPEC 2014-R71) obs
res_radec_2018_noMK = vcat(res_ras_2018_noMK, res_dec_2018_noMK) # α, δ residuals
W_radec_2018_noMK = W_radec_noMK # Farnocchia et al (2013) weighting scheme

;

In [ ]:
length(res_ras_2014), length(res_dec_2014)

In [ ]:
length(res_ras_2014_noMK), length(res_dec_2014_noMK)

In [ ]:
length(res_ras_2018), length(res_dec_2018)

In [ ]:
length(res_radec_2014_noMK), length(W_radec_2014_noMK), length(res_radec_2018), length(W_radec_2018)

In [ ]:
ans[3]-ans[2] #944-896

Compute time-delay/Doppler measurements residuals and weights:

In [ ]:
res_del = tdelay_jpl_obs .- vdel # delay residuals a.a.f. of (dx_{1,0},...,dx_{6,0},A2)
res_dop = dshift_jpl_obs .- vdop # Doppler residuals a.a.f. of (dx_{1,0},...,dx_{6,0},A2)
W_del = 1 ./(tdelay_jpl_obs_sigma.^2) # delay weights
W_dop = 1 ./(dshift_jpl_obs_sigma.^2) # Doppler weights

res_deldop = vcat(res_del, res_dop) # delay + Doppler residuals as a function of (dx_{1,0},...,dx_{6,0},A2)
W_deldop = vcat(W_del, W_dop) # delay + Doppler weights;

In [ ]:
length(res_del), length(res_dop), length(res_deldop)

Concatenate optical + radar residuals and weights:

In [ ]:
res_optrad_RAW = vcat(res_radec_RAW, res_deldop)
W_optrad_RAW = vcat(W_radec_RAW, W_deldop)

res_optrad_RAW_noMK = vcat(res_radec_RAW_noMK, res_deldop)
W_optrad_RAW_noMK = vcat(W_radec_RAW_noMK, W_deldop)

res_optrad_2014 = vcat(res_radec_2014, res_deldop)
W_optrad_2014 = vcat(W_radec_2014, W_deldop)

res_optrad_2014_noMK = vcat(res_radec_2014_noMK, res_deldop)
W_optrad_2014_noMK = vcat(W_radec_2014_noMK, W_deldop)

res_optrad_2018 = vcat(res_radec_2018, res_deldop)
W_optrad_2018 = vcat(W_radec_2018, W_deldop)

res_optrad_2018_noMK = vcat(res_radec_2018_noMK, res_deldop)
W_optrad_2018_noMK = vcat(W_radec_2018_noMK, W_deldop)
;

In [ ]:
length(res_optrad_2014), length(W_optrad_2014)

In [ ]:
length(res_optrad_2014_noMK), length(W_optrad_2014_noMK)

In [ ]:
length(res_optrad_2018), length(W_optrad_2018)

In [ ]:
length(res_optrad_2018_noMK), length(W_optrad_2018_noMK)

In [ ]:
# 7-DOF optical+radar fit, **NO** debias, **IN**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR7_RAW, Γ_OR7_RAW = newtonls(res_optrad_RAW, W_optrad_RAW, zeros(7), 5);
σ_OR7_RAW = sqrt.(diag(Γ_OR7_RAW))
@show x_OR7_RAW σ_OR7_RAW

# 7-DOF optical+radar fit, F2015 debias, **IN**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR7_2014, Γ_OR7_2014 = newtonls(res_optrad_2014, W_optrad_2014, zeros(7), 5);
σ_OR7_2014 = sqrt.(diag(Γ_OR7_2014))
@show x_OR7_2014 σ_OR7_2014

# 7-DOF optical+radar fit, E2020 debias, **IN**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR7, Γ_OR7 = newtonls(res_optrad_2018, W_optrad_2018, zeros(7), 4);
σ_OR7 = sqrt.(diag(Γ_OR7))
@show x_OR7 σ_OR7

x_OR7_RAW_noMK, Γ_OR7_RAW_noMK = newtonls(res_optrad_RAW_noMK, W_optrad_RAW_noMK, zeros(7), 5);
σ_OR7_RAW_noMK = sqrt.(diag(Γ_OR7_RAW_noMK))
@show x_OR7_RAW_noMK σ_OR7_RAW_noMK

x_OR7_2014_noMK, Γ_OR7_2014_noMK = newtonls(res_optrad_2014_noMK, W_optrad_2014_noMK, zeros(7), 5);
σ_OR7_2014_noMK = sqrt.(diag(Γ_OR7_2014_noMK))
@show x_OR7_2014_noMK σ_OR7_2014_noMK

x_OR7_noMK, Γ_OR7_noMK = newtonls(res_optrad_2018_noMK, W_optrad_2018_noMK, zeros(7), 9);
σ_OR7_noMK = sqrt.(diag(Γ_OR7_noMK))
@show x_OR7_noMK σ_OR7_noMK

##### OR6 no MK debiasing variations

x_OR6_RAW_noMK, Γ_OR6_RAW_noMK = newtonls_6v(res_optrad_RAW_noMK, W_optrad_RAW_noMK, zeros(7), 5);
σ_OR6_RAW_noMK = sqrt.(diag(Γ_OR6_RAW_noMK))
@show x_OR6_RAW_noMK σ_OR6_RAW_noMK

x_OR6_2014_noMK, Γ_OR6_2014_noMK = newtonls_6v(res_optrad_2014_noMK, W_optrad_2014_noMK, zeros(7), 5);
σ_OR6_2014_noMK = sqrt.(diag(Γ_OR6_2014_noMK))
@show x_OR6_2014_noMK σ_OR6_2014_noMK

x_OR6_noMK, Γ_OR6_noMK = newtonls_6v(res_optrad_2018_noMK, W_optrad_2018_noMK, zeros(7), 9);
σ_OR6_noMK = sqrt.(diag(Γ_OR6_noMK))
@show x_OR6_noMK σ_OR6_noMK
;

In [ ]:
chi2(res_optrad_2018(x_OR7), W_optrad_2018)

In [ ]:
chi2(res_optrad_RAW_noMK(x_OR7_RAW_noMK), W_optrad_RAW_noMK)

In [ ]:
chi2(res_optrad_2014_noMK(x_OR7_2014_noMK), W_optrad_2014_noMK)

In [ ]:
chi2(res_optrad_2018_noMK(x_OR7_noMK), W_optrad_2018_noMK)

In [ ]:
chi2(res_optrad_RAW_noMK(x_OR6_RAW_noMK), W_optrad_RAW_noMK)

In [ ]:
chi2(res_optrad_2014_noMK(x_OR6_2014_noMK), W_optrad_2014_noMK)

In [ ]:
chi2(res_optrad_2018_noMK(x_OR6_noMK), W_optrad_2018_noMK)

In [ ]:
#chi2(res_optrad_RAW(x_OR6_RAW), W_optrad_RAW)

In [ ]:
#chi2(res_optrad_2014(x_OR6_2014), W_optrad_2014)

In [ ]:
#chi2(res_optrad_2018_noMK(x_OR6_noMK), W_optrad_2018_noMK)

# O6

In [ ]:
# with Farnocchia et al (2013) weighting scheme
x_O6_2014, Γ_O6_2014 = newtonls_6v(res_radec_2014, W_radec_2014, zeros(7), 12);
x_O6, Γ_O6 = newtonls_6v(res_radec_2018, W_radec_2018, zeros(7), 12);
x_O6

In [ ]:
sqrt.(diag(Γ_O6_2014))

In [ ]:
sqrt.(diag(Γ_O6))

In [ ]:
nrms(res_radec_2014(x_O6_2014), W_radec_2014), chi2(res_radec_2014(x_O6_2014), W_radec_2014)

In [ ]:
nrms(res_radec_2018(x_O6), W_radec_2018), chi2(res_radec_2018(x_O6), W_radec_2018)

# O7

In [ ]:
# with Farnocchia et al (2013) weighting scheme
x_O7, Γ_O7 = newtonls(res_radec_2018, W_radec_2018, zeros(7), 10);
x_O7

In [ ]:
sqrt.(diag(Γ_O7)) # sigmas were unchanged after debiasing

# R (radar-only fit)

In [ ]:
#7-DOF radar-only fit
x_R, Γ_R = newtonls(res_deldop, W_deldop, zeros(7), 6);
#cholesky(Hermitian(Γ_R))
x_R

In [ ]:
sqrt.(diag(Γ_R))

In [ ]:
#***DIFF CORRECTIONS*** 7-DOF radar-only fit
#x_R_dc, Γ_R_dc = diffcorr(res_deldop, W_deldop, zeros(7), 6);
#cholesky(Hermitian(Γ_R))
#x_R_dc
#sqrt.(diag(Γ_R_dc))
#sqrt(inv(inv(Γ_R_dc)[7,7]))

In [ ]:
#1-D (A2) radar-only fit
x_1D, Γ_1D, C_1D = Apophis.newtonls_A2(res_deldop, W_deldop, vcat(x_OR7[1:6], 0.0), 8);
#cholesky(Hermitian(Γ_R))
x_1D

In [ ]:
Γ_R

In [ ]:
0.0029656916981060585

In [ ]:
sqrt(inv(inv(Γ_1D)[7,7]))

In [ ]:
sqrt(inv(inv(Γ_R)[7,7]))

In [ ]:
sqrt(inv(inv(Γ_OR7)[7,7]))

In [ ]:
0.0001809104

In [ ]:
0.0029650895230656773/0.0001809104

In [ ]:
sqrt(inv(C_1D[7,7]))

In [ ]:
sqrt(inv(inv(Γ_R)[7,7]))

In [ ]:
sqrt(inv(C_1D[7,7]))

In [ ]:
sqrt(inv(inv(Γ_1D)[7,7]))

In [ ]:
sqrt.(diag(Γ_1D))

In [ ]:
Γ_R

In [ ]:
inv(Γ_R)

In [ ]:
Γ_OR7

In [ ]:
inv(Γ_OR7)

In [ ]:
Γ_O7

In [ ]:
sqrt(inv(inv(Γ_O7)[7,7]))

In [ ]:
sqrt(inv(inv(Γ_OR7)[7,7]))

In [ ]:
sqrt(inv(inv(Γ_R)[7,7]))

In [ ]:
sqrt(inv(inv(Γ_1D)[7,7]))

# OR7
## TODO: initialize Newton method iterations with initial condition taken from `orbitdeterminator` output!

In [ ]:
# Differential corrections algo with TaylorN autodiff
# results are essentially the same as Newton method
# 7-DOF optical+radar fit, E2020 debias, **IN**CLUDING MAUNA KEA (MPEC) 2014-R71
@time x_OR7_dc, Γ_OR7_dc = diffcorr(res_optrad_2018, W_optrad_2018, zeros(7), 8);
σ_OR7_dc = sqrt.(diag(Γ_OR7_dc))
@show x_OR7_dc σ_OR7_dc;

In [ ]:
# 7-DOF optical+radar fit, **NO** debias, **IN**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR7_RAW, Γ_OR7_RAW = newtonls(res_optrad_RAW, W_optrad_RAW, zeros(7), 5);
σ_OR7_RAW = sqrt.(diag(Γ_OR7_RAW))
@show x_OR7_RAW σ_OR7_RAW
# 7-DOF optical+radar fit, F2015 debias, **IN**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR7_2014, Γ_OR7_2014 = newtonls(res_optrad_2014, W_optrad_2014, zeros(7), 5);
σ_OR7_2014 = sqrt.(diag(Γ_OR7_2014))
@show x_OR7_2014 σ_OR7_2014
# 7-DOF optical+radar fit, F2015 debias, **EX**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR7_2014_noMK, Γ_OR7_2014_noMK = newtonls(res_optrad_2014_noMK, W_optrad_2014_noMK, zeros(7), 5);
σ_OR7_2014_noMK = sqrt.(diag(Γ_OR7_2014_noMK))
@show x_OR7_2014_noMK σ_OR7_2014_noMK
# 7-DOF optical+radar fit, E2020 debias, **IN**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR7, Γ_OR7 = newtonls(res_optrad_2018, W_optrad_2018, zeros(7), 10);
σ_OR7 = sqrt.(diag(Γ_OR7))
@show x_OR7 σ_OR7
# 7-DOF optical+radar fit, E2020 debias, **EX**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR7_noMK, Γ_OR7_noMK = newtonls(res_optrad_2018_noMK, W_optrad_2018_noMK, zeros(7), 9);
σ_OR7_noMK = sqrt.(diag(Γ_OR7_noMK))
@show x_OR7_noMK σ_OR7_noMK
;

In [ ]:
#χ2
chi2(res_optrad_2018_noMK(x_OR7_noMK), W_optrad_2018_noMK) # use sqrt(N) ad hoc weighting for no MK fit (Farnocchia et al 2015)

In [ ]:
chi2(res_optrad_2014_noMK(x_OR7_2014_noMK), W_optrad_2014_noMK)

In [ ]:
x_OR7

In [ ]:
x_OR7_dc

In [ ]:
σ_OR7

In [ ]:
σ_OR7_dc

In [ ]:
nrms(res_optrad_2018(x_OR7), W_optrad_2018)

In [ ]:
#nrms(res_optrad_2018(x_OR7+cholesky((Γ_OR7)).L*3randn(7)), W_optrad_2018)

In [ ]:
@show σ_OR7_RAW
@show σ_OR7_2014
@show σ_OR7
;

In [ ]:
chi2(res_optrad_RAW(x_OR7_RAW), W_optrad_RAW)

In [ ]:
chi2(res_optrad_2014(x_OR7_2014), W_optrad_2014)

In [ ]:
chi2(res_optrad_2018(x_OR7), W_optrad_2018)

In [ ]:
chi2(res_optrad_2014_noMK(x_OR7_2014_noMK), W_optrad_2014_noMK)

In [ ]:
chi2(res_optrad_2018_noMK(x_OR7_noMK), W_optrad_2018_noMK)

# OR6

In [ ]:
# 6-DOF optical+radar fit, E2020 debias, **IN**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR6_RAW, Γ_OR6_RAW = newtonls_6v(res_optrad_RAW, W_optrad_RAW, zeros(7), 10);
@show x_OR6_RAW
# 6-DOF optical+radar fit, F2015 debias, **EX**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR6_2014, Γ_OR6_2014 = newtonls_6v(res_optrad_2014, W_optrad_2014, zeros(7), 10);
@show x_OR6_2014
# 6-DOF optical+radar fit, E2020 debias, **IN**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR6, Γ_OR6 = newtonls_6v(res_optrad_2018, W_optrad_2018, zeros(7), 10);
x_OR6

In [ ]:
# 6-DOF optical+radar fit, E2020 debias, **IN**CLUDING MAUNA KEA (MPEC) 2014-R71
x_OR6_noMK, Γ_OR6_noMK = newtonls_6v(res_optrad_2018_noMK, W_optrad_2018_noMK, zeros(7), 10);
x_OR6_noMK

In [ ]:
chi2(res_optrad_RAW(x_OR6_RAW), W_optrad_RAW)

In [ ]:
chi2(res_optrad_2014(x_OR6_2014), W_optrad_2014)

In [ ]:
chi2(res_optrad_2018(x_OR6), W_optrad_2018)

In [ ]:
chi2(res_optrad_2018_noMK(x_OR6_noMK), W_optrad_2018_noMK)

In [ ]:
sqrt.(diag(Γ_OR6))

In [ ]:
for (sol,gamma, label) in [(x_O7, Γ_O7, "O7"), (x_O6, Γ_O6, "O6"), (x_OR7, Γ_OR7, "OR7"), (x_OR6, Γ_OR6, "OR6"), (x_R, Γ_R, "R")]
    println("*****", label)
    @show Apophis.initialcond(sol.*vcat(1e-8ones(6), 1e-14))
    @show sqrt.(diag(gamma)).*(vcat(1e-8ones(6), 1e-14)[1:size(gamma)[1]])
    println("*****")
    println(" ")
end

In [ ]:
### OR7

-0.9633018(236) \pm 4.00
 0.5100291(225) \pm 7.73
 0.1652802(958) \pm 3.05
-0.00711887(437) \pm 1.27
-0.012061234(331) \pm 7.37
-0.004669513(808) \pm 4.29
-4.97 \pm 2.74

3.995423593896419e-8
7.727487292244818e-8
3.048351693923244e-8
1.2677191394342694e-9
7.373269962552543e-10
4.293800535951439e-10
2.7414996740681604e-14




### O7


-0.9633018(387) \pm 7.88
0.510029(145) \pm 1.08
0.1652803(301) \pm 5.65
-0.00711887(482) \pm 1.99
-0.01206123(338) \pm 1.18
-0.004669514(163) \pm 6.67
-6.50 \pm 5.77

7.877625785328185e-8
1.083430271779183e-7
5.6541777392004144e-8
1.986184668146816e-9
1.1800699083640426e-9
6.671765805708299e-10
5.769676338572042e-14



### OR6

-0.96330189(585) \pm 2.87
0.51002898(279) \pm 4.88
0.16528024(301) \pm 8.91
-0.0071188720(846) \pm 6.91
-0.012061235(629) \pm 1.75
-0.004669514(385) \pm 2.88

2.865633094899382e-9
4.879584005625289e-9
8.906991910708259e-9
6.903886114569203e-11
1.7509927240644402e-10
2.8780910639953976e-10

### O6

-0.9633018(960) \pm 6.02
0.5100290(372) \pm 4.95
0.1652802(868) \pm 4.15
-0.007118872(877) \pm 9.74
-0.012061234(528) \pm 6.00
-0.004669514(3885) \pm 6.37

#6.018774193053022e-8
#4.945528838893412e-8
#4.149724900817595e-8
#9.735472239107363e-10
#5.990424976089863e-10
#6.367079180984414e-10



# `Taylor1` jet transport: Parametrized orbit determination

In this section we describe a parametrized orbit determination scheme exploiting jet transport techniques. Concretely, we seek to compute a 1-parameter family of 6-DOF nominal solutions $x^\star$ as a function of $A_2$. This is similar to what Milani et al. (2009) did something for asteroid (101955) Bennu. We may think that, for example, our 6-DOF (i.e., gravity-only) orbit and our 7-DOF orbit (i.e., including non-gravitational acceleration) for Apophis may be interpreted as two points in this parametrized scheme: the 6-DOF solution corresponds to $A_2=0$, whereas the 7-DOF solution corresponds to $A_2=-5.59... \times 10^{-14}$au/day$^2$. Thus, we are trying to find a parametrization of these two nominal solutions with respect to the non-gravitational transverse parameter $A_2$ which joins them.

We proceed as follows: first, we retrieve our 6-DOF nominal solution
$$
\mathbf{x}^\star_0 = \begin{pmatrix} x_0 \\
y_0 \\
z_0 \\
u_0 \\
v_0 \\
w_0 \\
0 \end{pmatrix}
$$

that is, the output of the orbital fit fixing $A_2=0$. Then, we add to the last component $A_2$ as a $p$-th order `Taylor1` variable to form the vector $\hat{\mathbf{x}}$:

$$
\hat{\mathbf{x}} = \begin{pmatrix} x_0 \\
y_0 \\
z_0 \\
u_0 \\
v_0 \\
w_0 \\
A_2 + O(A_2^{p+1}) \end{pmatrix}
$$

Now, we perform a 6-DOF orbit determination using the above initial condition. We denote the output of this procedure as $\mathbf{x}^\star(A_2)$, which is a 1-parameter family of orbital fits wrt the Yarkovsky parameter $A_2$:

$$
\mathbf{x}^\star(A_2) = \begin{pmatrix} x_0+x_1A_2+\ldots+x_{p}A_2^{p}+ O(A_2^{p+1}) \\
y_0+y_1A_2+\ldots+y_{p}A_2^{p}+ O(A_2^{p+1}) \\
z_0+z_1A_2+\ldots+z_{p}A_2^{p}+ O(A_2^{p+1}) \\
u_0+u_1A_2+\ldots+u_{p}A_2^{p}+ O(A_2^{p+1}) \\
v_0+v_1A_2+\ldots+v_{p}A_2^{p}+ O(A_2^{p+1}) \\
w_0+w_1A_2+\ldots+w_{p}A_2^{p}+ O(A_2^{p+1}) \\
A_2 + O(A_2^{p+1}) \end{pmatrix}
$$

The coefficients $x_0,x_1,\ldots,y_0,y_1,\ldots\,w_{p}$ are the Taylor coefficients of the orbital fit $\mathbf{x}^\star_0$ wrt $A_2$. Using this expression, we are able to evaluate the post-fit RMS (and hence its minimum) as a function of $A_2$. This parametrized orbit determination scheme is self-consistent in the sense that if we evaluate $\mathbf{x}^\star(A_2)$ with $A_2=0$, then we recover the 6-DOF nominal solution $\mathbf{x}^\star_0$, and if we evaluate $\mathbf{x}^\star(A_2)$ with $A_2=A_2^\star$, then we recover the 7-DOF orbital fit.

Even as we are performing a 6-DOF least squares procedure, the covariance matrix associated to the parametrized orbital fit $\mathbf{x}^\star_0$ wrt $A_2$ will be 7x7. That is, this scheme allow us to compute the 1-$\sigma$ formal uncertainty associated to a given value of $A_2$.

We may now use this 1-parameter family of nominal initial conditions to propagate the orbit of Apophis as a function of an unknown $A_2$, taking into account correlations between $A_2$ and the initial conditions. Using this, we may obtain predictions as functions of $A_2$, and thus we may be able to determine $A_2$ from new observations. In ra/dec plots or delay/doppler prediction plots, we may plot the nominal prediction, the 3-$\sigma$ covariance matrix and the line corresponding to several distinct values of $A_2$. It would be also possible to propagate $\mathbf{x}^\star(A_2)$ under the equations of motion of Apophis with 1-variable jet transport techniques to assess the impact hazard of Apophis as a function of $A_2$, say, for the next 100 years. In this sense, this parametrized orbit determination scheme is equivalent to a line of variations (LOV) defined wrt the parameter $A_2$.

In [ ]:
#x0T1 = x0_JPL + vcat(zeros(6), Taylor1(10))
x0T1 = x_OR6 + vcat(zeros(6)*Taylor1(11), Taylor1(11))

In [ ]:
xT1, ΓT1 = newtonls_6v(res_optrad_2018, W_optrad_2018, deepcopy(x0T1), 4);

In [ ]:
xT1

In [ ]:
dqT1 = vcat(1e-8ones(6), 1e-14).*xT1

In [ ]:
Apophis.initialcond(dqT1(A2_min)) .- Apophis.initialcond(dqT1())

In [ ]:
dqT1(A2_min) .- dqT1()

In [ ]:
vcat(1e-8ones(6), 1e-14).*(x_OR7 .- x_OR6)

In [ ]:
dqT1

In [ ]:
dqT1()

In [ ]:
dqT1(A2_min)

In [ ]:
jldopen("dqT1_LOV_A2.jld", "w") do file
    write(file, "dqT1", dqT1)
end

In [ ]:
xT1

In [ ]:
xT1-x0T1

In [ ]:
rms_A2 = nrms(res_optrad_2018(xT1), W_optrad_2018)

In [ ]:
A2_min = Apophis.taylor_minimum(rms_A2, 0.0)

In [ ]:
xT1() #- x_OR6

In [ ]:
xT1(A2_min) #- x_OR7

In [ ]:
xT1(A2_JPL)

In [ ]:
#map(x->x[0], ΓT1)

In [ ]:
#map(x->x[1], ΓT1)

In [ ]:
#map(x->x[end], ΓT1)

In [ ]:
sqrt.(diag(ΓT1))

In [ ]:
sqrt.(diag(ΓT1(A2_min)))

In [ ]:
sqrt.(diag(ΓT1()))

In [ ]:
A2v = -15:0.01:5

In [ ]:
using Plots

In [ ]:
plot(A2v, rms_A2.(A2v), leg=false)
xlabel!("A₂ [×10^{14} au / d^{2} ]")
ylabel!("normalized RMS")

In [ ]:
nrms(res_optrad_2018(x_OR6), W_optrad_2018)

In [ ]:
nrms(res_optrad(x_optrad), W_optrad)
#was 0.5104665279115688 before debiasing

In [ ]:
nrms(res_optrad(xT1()), W_optrad)
# was 0.5108022792732124 before debiasing

In [ ]:
nrms(res_optrad(xT1(A2_min)), W_optrad)
# was 0.5104665279115705 before debiasing

# High-order Taylor parametrization of the Line of Variations

According to Milani (1999, 2005, etc.), there are several ways to define the line of variations (LOV). For example, following Milani (2005), we could define the LOV in terms of the "weak" direction defined by the largest eigenvalue of the covariance matrix $\Gamma$. Here, we describe how to numerically compute and propagate a high-order Taylor parametrization of the LOV for several choices of the definition of the LOV through an ordinary differential equation. Our goal is to construct a parametrization of the LOV, which is valid outside the linear regime, e.g., up to 5-sigma (Milani, 2005), which we propagate in time exploiting jet transport techniques for impact hazard assessment. The material presented in this section is based on the parameterization method, which has been used to compute parameterizations of stable/unstable manifolds associated to fixed points of discrete and continuous dynamical systems (Mireles-James, Haro, etc.).

Given a nominal solution $\mathbf{x}^\star$, the LOV $X(\sigma)$ may be defined via the differential equation
$$
\frac{d\mathbf{X}(\sigma)}{d\sigma} = \mathbf{F}(\mathbf{X}(\sigma))
$$
subject to the initial condition $\mathbf{X}(\sigma=0)=\mathbf{x}^\star$, where $\sigma$ is the parameter of the LOV, and $\mathbf{F}$ is a vector field. Distinct choices for the vector field $\mathbf{F}$ correspond to distinct definitions of the LOV. For example, we may choose $\mathbf{F}=-\frac{\partial Q}{\partial \mathbf{X}}$, where $Q$ is the weighted mean squared error, i.e., the target function of the least squares process. This choice corresponds to a _steepest descent_ definition of the LOV.

In order to obtain a high-order Taylor parametrization of the LOV, we propose a solution to the ODE above as a power series in $\sigma$:
$$
\mathbf{X}(\sigma) = \sum_{n=0}^{\infty}X_n \sigma^n,
$$
where each coefficient $X_n$ represents the $n$-th Taylor coefficient of $X$ at $\sigma=0$. We see that the $0$-th order coefficients $X_0$ correspond to the initial condition (i.e., the nominal solution). On the other hand, in order to construct the LOV, we impose an additional condition, which is to set the first-order coefficients $X_1$ equal to the eigenvector associated to the weak direction, defined as the direction of the eigenvector associated to the smallest eigenvalue of the normal matrix $C$. This corresponds to the construction of the LOV in the linear regime.

For $n\geq2$, we may compute the Taylor coefficients $X_n$ of the LOV as follows. Substituting the power series ansatz into the LOV ODE, and equating equal power coeffients, we arrive at the usual Taylor recursion formula for ODEs:
$$
X_{n+1} = \frac{F_n}{n+1}.
$$

In [ ]:
x_sol_ = x_optrad
Γ_sol_ = Γ_optrad

In [ ]:
# jet transport propagation scaling
jt_scaling = vcat(1e-8ones(6), 1e-14)
dofs = 7 #size(Γ_sol_)[1]
# Compute nominal initial condition in au, au/day, au/day^2
q0_optrad = Apophis.initialcond(jt_scaling.*x_sol_)
# relative scalings analogous to Milani (2005)
scaling_milani_2005 = vcat(  (1/norm(q0_optrad[1:3]))*ones(3), (1/norm(q0_optrad[4:6]))*ones(3), 1/norm(q0_optrad[7])  )
# compute cov matrix with relative scalings
#Γ_sol_scm05 = diagm(scaling_milani_2005) * ( diagm(jt_scaling)*Γ_sol_*diagm(jt_scaling) ) * diagm(scaling_milani_2005)
Γ_sol_scm05 = diagm(scaling_milani_2005[1:dofs]) * ( diagm(jt_scaling[1:dofs])*Γ_sol_[1:dofs,1:dofs]*diagm(jt_scaling[1:dofs]) ) * diagm(scaling_milani_2005[1:dofs])
# compute normal matrix with relative scalings
C_sol_scm05 = inv(Γ_sol_scm05)
# compute eigenvalues, eigenvectors of normal matrix with relative scalings
C_eig_sol_scm05 = eigen(C_sol_scm05)
# revert eigenvector of "weak" direction to JT scaling
v_weak = C_eig_sol_scm05.vectors[:,1]./scaling_milani_2005[1:dofs]./jt_scaling[1:dofs]

In [ ]:
C_sol_ = inv(Γ_sol_)

In [ ]:
C_eig_sol_ = eigen(C_sol_)

In [ ]:
norm(v_weak)

In [ ]:
#x0T2 = x_sol_ .+ C_eig_sol_.vectors[:,1].*(1e-6Taylor1(5))
x0T2 = x_sol_ .+ (v_weak/norm(v_weak)).*(Taylor1(6))

In [ ]:
# adapted from TaylorIntegration.jetcoeffs!
function lovcoeffs!(eqsdiff!::Function, t::Taylor1{T},
        x::AbstractVector{Taylor1{U}}, dx::AbstractVector{Taylor1{U}},
        xaux::AbstractVector{Taylor1{U}}, params) where {T<:Real, U<:Number}
    order = x[1].order
    for ord in 1:order-1 # since order 0 and 1 are set, we start at ordnext=2 (i.e., ord=1)
        ordnext = ord+1

        # Set `taux`, auxiliary Taylor1 variable to order `ord`
        @inbounds taux = Taylor1( t.coeffs[1:ordnext] )
        # Set `xaux`, auxiliary vector of Taylor1 to order `ord`
        for j in eachindex(x)
            @inbounds xaux[j] = Taylor1( x[j].coeffs[1:ordnext+1] )
        end

        # Equations of motion
        eqsdiff!(dx, xaux, params, taux)

        # Recursion relations
        #@show x
        #@show dx
        for j in eachindex(x)
            #@show j
            @inbounds x[j][ordnext] = dx[j][ord]/ordnext
        end
    end
    nothing
end

In [ ]:
chi2(res_optrad, W_optrad).order

In [ ]:
Q__ = chi2(res_optrad, W_optrad) # sum(res_optrad .* (res_optrad.^2))/length(res_optrad);

In [ ]:
Q__(x_optrad)

In [ ]:
dQ__ = TaylorSeries.gradient(Q__);

In [ ]:
dQ__()

In [ ]:
dQ__(x0_JPL)

In [ ]:
dQ__(x_optrad)

In [ ]:
dQ__(x0T2)

In [ ]:
function mdQ(dx, x, params, t)
    #@show x
    dQ = dQ__(x)
    #@show dQ
    for i in 1:7
        dx[i] = -dQ[i]
    end
    return nothing
end

In [ ]:
t = Taylor1(x0T2[1].order)
dx__ = similar(x0T2)
dx__ .= Taylor1.(0.0, t.order)
xaux__ = similar(x0T2);

In [ ]:
lovcoeffs!(mdQ, t, x0T2, dx__, xaux__, nothing);

In [ ]:
x0T2

In [ ]:
σ_lim = 2e-6
σv = range(-σ_lim, length=200, stop=σ_lim)

In [ ]:
plot(
    σv, x0T2[1].(σv).-x_optrad[1]
)
plot!(
    σv, x0T2[2].(σv).-x_optrad[2]
)
plot!(
    σv, x0T2[3].(σv).-x_optrad[3]
)

In [ ]:
plot(
    σv, x0T2[4].(σv).-x_optrad[4]
)
plot!(
    σv, x0T2[5].(σv).-x_optrad[5]
)
plot!(
    σv, x0T2[6].(σv).-x_optrad[6]
)

In [ ]:
plot(
    σv, x0T2[7].(σv)#.-x_optrad[7]
)

In [ ]:
norm(dQ__(x0T2(-2e-2)))

In [ ]:
# initial condition
x0T2() == x_optrad

In [ ]:
# 1st order coeffs == eigenvector
#1e-6C_eig_sol_.vectors[:,1] == map(x->x[1], x0T2)
v_weak/norm(v_weak) == map(x->x[1], x0T2)

In [ ]:
x0T2(σv[1])

In [ ]:
x0T2(σv[end])

In [ ]:
norm(dQ__(x0T2(σv[1])))

In [ ]:
norm(dQ__(x0T2(σv[end])))

In [ ]:
norm(dQ__(x0T2()))

In [ ]:
TaylorIntegration.stepsize(x0T2, 1e-16)

# Post-fit normalized RMS

In [ ]:
# 7-DOF optical+radar residuals evaluated at optical+radar nominal solution, Farnocchia et al (2013) weights
nrms(res_optrad_2018(x_OR7), W_optrad_2018), nrms(res_optrad_2018(x_OR7), W_optrad_2018)^2

In [ ]:
# 7-DOF optical+radar residuals evaluated at optical+radar nominal solution, Farnocchia et al (2013) weights
nrms(res_optrad(x_optrad), W_optrad), nrms(res_optrad(x_optrad), W_optrad)^2

In [ ]:
# 6-DOF (A2=0) optical+radar residuals evaluated at optical+radar nominal solution, Farnocchia et al (2013) weights
nrms(res_optrad(x_optrad_6v), W_optrad)

In [ ]:
# optical+radar residuals evaluated at optical+radar nominal solution, naive weights
nrms(res_optrad(x_optrad), W_optrad)

In [ ]:
# optical+radar residuals evaluated at JPL#199 nominal solution, Farnocchia et al (2013) weights
nrms(res_optrad(x0_JPL), W_optrad)

In [ ]:
# optical+radar residuals evaluated at JPL#199 nominal solution, naive weights
nrms(res_optrad(x0_JPL), W_optrad)

In [ ]:
# optical-only residuals evaluated at optical-only nominal solution, Farnocchia et al (2013) weights
nrms(res_radec(x_opt), W_radec)

In [ ]:
# optical-only residuals evaluated at optical-only nominal solution, naive weights
nrms(res_radec(x_opt), W_radec)

In [ ]:
# optical-only residuals evaluated at JPL#199 nominal solution, Farnocchia et al (2013) weights
nrms(res_radec(x0_JPL), W_radec)

In [ ]:
# optical-only residuals evaluated at JPL#199 nominal solution, naive weights
rms(res_radec(x0_JPL), W_radec)

In [ ]:
# radar-only residuals evaluated at radar-only nominal solution
nrms(res_deldop(x_rad), W_deldop)

In [ ]:
# radar-only residuals evaluated at JPL#199 nominal solution
nrms(res_deldop(x0_JPL), W_deldop)

## Note:

$$
Q(x) = \frac{1}{m}\xi^T W \xi
$$

We have
$$
\frac{\partial Q}{\partial x} = \frac{2}{m}(B^T W \xi),
$$
$$
\frac{\partial^2 Q}{\partial x^2} = \frac{2}{m}(B^T W B + W \xi^T H)
$$
and
$$
C = B^T W B.
$$

$$
\Gamma = C^{-1}
$$
Thus, the second derivative $\frac{\partial^2 Q}{\partial x^2} \neq \frac{2}{m}C$, if we include the $W \mathbf{\xi}^T H$ term (Milani and Gronchi, 2010, Eq. 5.3). Nevertheless, $\frac{\partial^2 Q}{\partial x^2}$ value is identical, up to `eps(Float64)`, when computing it either via `TaylorSeries.hessian` or with Milani's formulae.

$$
\frac{1}{x+a}=\frac{1}{a}\frac{1}{1+x/a} \sim \frac{1}{a}(1-x/a)
$$

# Plots

## Optical astrometry residuals

In [ ]:
using Plots

In [ ]:
const t0 = datetime2julian(DateTime(2008,9,24))

In [ ]:
scatter(tdelay_jpl_obs_jd_utc[3:end].-t0, res_del(x_OR7)[3:end], yerror=tdelay_jpl_obs_sigma[3:end])
title!("Time delay residuals vs time (2012-2013)")
ylabel!("Time delay [us]")
xlabel!("Date")

xlims!(1540, 1640)
xticks!((-1250):0.075yr:(2100), string.(Date.(julian2datetime.(t0 .+ collect((-1250):0.075yr:(2100))))))

In [ ]:
scatter(dshift_jpl_obs_jd_utc[6:end].-t0, res_dop(x_OR7)[6:end], yerror=dshift_jpl_obs_sigma[6:end])
title!("Doppler shift residuals vs time (2012-2013)")
ylabel!("Doppler shift [Hz]")
xlabel!("Date")

xlims!(1540, 1640)
xticks!((-1250):0.075yr:(2100), string.(Date.(julian2datetime.(t0 .+ collect((-1250):0.075yr:(2100))))))

In [ ]:
scatter(
    vcat(tv_opt, tv_opt_v15)[no_mk_ind_t].-t0, 
    res_ras_2018_noMK(x_OR7_noMK),
    yerror=sqrt.(1 ./vcat(w_α_f13, w_α_v15))[no_mk_ind_t],
    label="post-fit resd. (OR7)"
)
title!("Right ascension residuals vs time")
ylabel!("Right ascension [arcseconds]")
xlabel!("Calendar date")
xlims!(-1250, 2100)
xticks!((-1250):2yr:(2100), string.(Date.(julian2datetime.(t0 .+ collect((-1250):2yr:(2100))))))

In [ ]:
scatter(
    vcat(tv_opt, tv_opt_v15)[no_mk_ind_t].-t0,
    res_dec_2018_noMK(x_OR7_noMK),
    yerror=sqrt.(1 ./vcat(w_δ_f13, w_δ_v15))[no_mk_ind_t],
    label="post-fit resd. (OR7)"
)
title!("Declination residuals vs time")
ylabel!("Declination [arcseconds]")
xlabel!("Calendar date")
xlims!(-1250, 2100)
xticks!((-1250):2yr:(2100), string.(Date.(julian2datetime.(t0 .+ collect((-1250):2yr:(2100))))))

In [ ]:
#histogram(res_ras(x0_JPL))
#histogram!(res_ras(x0_newton))
histogram(res_ras_2018(x_OR7_noMK)[1:432]) # should include weights somehow?
xlabel!("R.A. residuals [arcseconds]")

In [ ]:
mean(res_ras_2018(x_OR7_noMK)[1:432]), std(res_ras_2018(x_OR7_noMK)[1:432])

In [ ]:
histogram(res_dec_2018(x_OR7_noMK)[1:432])
#histogram!(res_dec())
xlabel!("Declination residuals [arcseconds]")

In [ ]:
mean(res_dec_2018(x_OR7_noMK)[1:432]), std(res_dec_2018(x_OR7_noMK)[1:432])

In [ ]:
# evaluate jet transport residuals at corresponding solution
res_α_OR7_RAW = res_ras_RAW(x_OR7_RAW)
res_δ_OR7_RAW = res_dec_RAW(x_OR7_RAW);

res_α_OR7_2014 = res_ras_2014(x_OR7_2014)
res_δ_OR7_2014 = res_dec_2014(x_OR7_2014)

res_α_OR7_2018 = res_ras_2018(x_OR7)
res_δ_OR7_2018 = res_dec_2018(x_OR7)
;

In [ ]:
cov_αδ_RAW = cov(map((x,y)->[x,y], res_α_OR7_RAW[1:432], res_δ_OR7_RAW[1:432]))

In [ ]:
cov_αδ_2014 = cov(map((x,y)->[x,y], res_α_OR7_2014[1:432], res_δ_OR7_2014[1:432]))

In [ ]:
cov_αδ_2018 = cov(map((x,y)->[x,y], res_α_OR7_2018[1:432], res_δ_OR7_2018[1:432]))

In [ ]:
cov_αδ_cholf_RAW = cholesky(cov_αδ_RAW)

In [ ]:
cov_αδ_cholf_2014 = cholesky(cov_αδ_2014)

In [ ]:
cov_αδ_cholf_2018 = cholesky(cov_αδ_2018)

In [ ]:
sigma_factor = 3

In [ ]:
angv = vcat(range(0, length=1000, stop=prevfloat(2pi)), 0);

In [ ]:
σ_cos_ang = sigma_factor*cos.(angv)
σ_sin_ang = sigma_factor*sin.(angv);

In [ ]:
ell_3σ_x_RAW = cov_αδ_cholf_RAW.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_RAW.L[1,2].*σ_sin_ang
ell_3σ_y_RAW = cov_αδ_cholf_RAW.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_RAW.L[2,2].*σ_sin_ang;

ell_3σ_x_2014 = cov_αδ_cholf_2014.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_2014.L[1,2].*σ_sin_ang
ell_3σ_y_2014 = cov_αδ_cholf_2014.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_2014.L[2,2].*σ_sin_ang;

ell_3σ_x_2018 = cov_αδ_cholf_2018.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_2018.L[1,2].*σ_sin_ang
ell_3σ_y_2018 = cov_αδ_cholf_2018.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_2018.L[2,2].*σ_sin_ang;

In [ ]:
mean(res_α_OR7_RAW), mean(res_δ_OR7_RAW)

In [ ]:
mean(res_α_OR7_2014), mean(res_δ_OR7_2014)

In [ ]:
mean(res_α_OR7_2018), mean(res_δ_OR7_2018)

In [ ]:
mean(res_α_OR7_RAW[1:432]), mean(res_δ_OR7_RAW[1:432])

In [ ]:
mean(res_α_OR7_2014[1:432]), mean(res_δ_OR7_2014[1:432])

In [ ]:
mean(res_α_OR7_2018[1:432]), mean(res_δ_OR7_2018[1:432])

In [ ]:
using Gnuplot

In [ ]:
Gnuplot.options.gpviewer = false

In [ ]:
obs_ind = 1:432

@gp "unset grid" :-
#@gp :- "set si sq" :-
#@gp :- "set size ratio -1" :-
#@gp :- "set key outside top right" :-
@gp :- "set xtics 0.5" :-
@gp :- "set ytics 0.1" :-
@gp :- xrange=[-0.5,1.0] :-
@gp :- yrange=[-0.5,0.5] :-
@gp :- "set xlabel '\$\\alpha\\cos\\delta\$ [arcsec]'" :-
@gp :- "set ylabel '\$\\delta\$ [arcsec]'" :-

@gp :- "set style fill transparent solid 0.35 noborder" :-
@gp :- "set style circle radius 0.015" :-

@gp :- res_α_OR7_RAW[obs_ind] res_δ_OR7_RAW[obs_ind]   "w p ps 1.5 lw 3 t 'OR7, no debiasing'" :-
@gp :- res_α_OR7_2014[obs_ind] res_δ_OR7_2014[obs_ind] "w p ps 1.5 lw 3 t 'OR7, debiased (Farnocchia et al., 2015)'" :-
@gp :- res_α_OR7_2018[obs_ind] res_δ_OR7_2018[obs_ind] "w p ps 1.5 lw 3 t 'OR7, debiased (Eggl et al., 2020)'" :-

@gp :- mean(res_α_OR7_RAW[obs_ind]).+ell_3σ_x_RAW mean(res_δ_OR7_RAW[obs_ind]).+ell_3σ_y_RAW "w l lw 3 lt 1 not" :-
@gp :- mean(res_α_OR7_2014[obs_ind]).+ell_3σ_x_2014 mean(res_δ_OR7_2014[obs_ind]).+ell_3σ_y_2014 "w l lw 3 lt 2 not" :-
@gp :- mean(res_α_OR7_2018[obs_ind]).+ell_3σ_x_2018 mean(res_δ_OR7_2018[obs_ind]).+ell_3σ_y_2018 "w l lw 3 lt 3 not" :-

@gp :- mean(res_α_OR7_RAW[obs_ind]) mean(res_δ_OR7_RAW[obs_ind]) "w circle lt 1 fs solid 1.0 border lt -1 not" :-
@gp :- mean(res_α_OR7_2014[obs_ind]) mean(res_δ_OR7_2014[obs_ind]) "w circle lt 2 fs solid 1.0 border lt -1 not" :-
@gp :- mean(res_α_OR7_2018[obs_ind]) mean(res_δ_OR7_2018[obs_ind]) "w circle lt 3 fs solid 1.0 border lt -1 not"

#Gnuplot.save(term="epslatex standalone color dashed size 6in,5in font \",12\"", output="debias.tex")
#run(`pdflatex debias.tex`)

## Radar astrometry residuals

In [ ]:
###time-delay and Doppler residuals from Brozovic et al., 2018
###includes both radar astrometric datasets: 2005-2006 and 2012-2013
b2018_td_res = [0.273, 0.313, 0.105, 0.023, -0.491, 0.018, -0.123, -0.599, 0.121, 0.072, 0.056, 0.021, 0.059, -0.374, -0.154, 0.049, 0.400]
b2018_ds_res = [-0.049, 0.017, 0.058, -0.098, -0.006, 0.039, -0.023, -0.001, 0.107, -0.061, -0.052, 0.037, -0.009, -0.007, -0.027, 0.031, 0.113, -0.062, 0.010, -0.077, 0.070, -0.006, 0.009, -0.068, 0.004, 0.023, 0.016, 0.029, 0.150]
;

In [ ]:
#scatter(
#    tdelay_jpl_obs_jd_utc.-t0, 
#    res_del(x0_JPL), 
#    marker=:square,
#    ms=4.0, #msc=:auto,
#    yerror = tdelay_jpl_obs_sigma, 
#    label="pre-fit (current)"
#)
scatter(
    tdelay_jpl_obs_jd_utc.-t0, 
    b2018_td_res, 
    marker=:square,
    ms=4.0, #msc = :auto,
    yerror = tdelay_jpl_obs_sigma, 
    label="JPL#197 (Brozovic et al., 2018)",
    legend=:topleft,
    annotations=( -1332.75, 6, "(a)")
)
scatter!(
    tdelay_jpl_obs_jd_utc.-t0, 
    res_del(x_OR7),
    marker=:circle,
    ms=4.0, #msc=:auto,
    yerror = tdelay_jpl_obs_sigma, 
    label="post-fit (current)"
)
title!("Time delay residuals vs time")
ylabel!("Time delay (us)")
xlabel!("Date")
xlims!(1540, 1645)
xticks!(1540:20:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:20:1640)))))

In [ ]:
t0td = tdelay_jpl_obs_jd_utc[1]
p1a = scatter(
    tdelay_jpl_obs_jd_utc.-t0, 
    b2018_td_res, 
    marker=:square,
    ms=3.0,
    yerror = tdelay_jpl_obs_sigma, 
    label="JPL#197",
    legend=:topleft,
    annotations=( -1332.75, 6, "(a)")
)
scatter!(
    tdelay_jpl_obs_jd_utc.-t0, 
    res_del(x_OR7), 
    marker=:circle, 
    yerror = tdelay_jpl_obs_sigma, 
    label="OR7"
)
ylims!(-5.1,8)
xlims!(-1334.5, -1332.5)
xticks!((-1334.5):0.5:(-1332.5), string.(Date.(julian2datetime.(t0 .+ collect((-1334.5):0.5:(-1332.5))))))
#title!("Post-fit time delay residuals (O-C): 2005-2006")
ylabel!("Time delay resid. (O-C) [us]")
xlabel!("Date")

In [ ]:
t0td = tdelay_jpl_obs_jd_utc[1]
p1b = scatter(
    tdelay_jpl_obs_jd_utc.-t0, 
    b2018_td_res, 
    marker=:square,
    ms=3.0,
    yerror = tdelay_jpl_obs_sigma, 
    label="JPL#197",
    legend=:topleft,
    annotations=( 1630, 4, "(b)")
)
scatter!(
    tdelay_jpl_obs_jd_utc.-t0, 
    res_del(x_OR7), 
    marker=:circle, 
    yerror = tdelay_jpl_obs_sigma, 
    label="OR7"
)
#title!("Post-fit time delay residuals (O-C): 2012-2013")
ylabel!("Time delay resid. (O-C) [us]")
xlims!(1540, 1645)
xticks!(1540:30:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:30:1640)))))
xlabel!("Date")

In [ ]:
plot(p1a, p1b, size=(640,860), layout=@layout([a; b]))
#savefig("time_delay_residuals__.pdf")
#savefig("~/phdthesis/fig5-7.pdf")

In [ ]:
t0ds = dshift_jpl_obs_jd_utc[1]
p2a = scatter(
    dshift_jpl_obs_jd_utc.-t0, 
    b2018_ds_res, 
    marker=:square,
    ms=3.0,
    yerror = dshift_jpl_obs_sigma, 
    label="JPL#197",
    annotations=( -925, 0.4, "(c)")
)
scatter!(
    dshift_jpl_obs_jd_utc.-t0, 
    res_dop(x_OR7), 
    marker=:circle, 
    yerror = dshift_jpl_obs_sigma, 
    label="OR7",
    leg=:topleft
)
ylims!(-0.4,0.5)
xlims!(-1350,-850)
xticks!((-1350):150:(-850), string.(Date.(julian2datetime.(t0 .+ collect((-1350):150:(-850))))))
#title!("Post-fit Doppler shift residuals (O-C): 2005-2006")
ylabel!("Doppler shift resid. (O-C) [Hz]")
xlabel!("Date")

In [ ]:
t0ds = dshift_jpl_obs_jd_utc[1]
p2b = scatter(
    dshift_jpl_obs_jd_utc.-t0, 
    b2018_ds_res, 
    marker=:square,
    ms=3.0,
    yerror = dshift_jpl_obs_sigma, 
    label="JPL#197",
    leg=:topright,
    annotations=( 1630, 0.5, "(d)")
)
scatter!(
    dshift_jpl_obs_jd_utc.-t0, 
    res_dop(x_OR7), 
    marker=:circle, 
    yerror = dshift_jpl_obs_sigma, 
    label="OR7",
    leg=:topleft
)
#title!("Post-fit Doppler shift residuals (O-C): 2012-2013")
ylabel!("Doppler shift resid. (O-C) [Hz]")
xlabel!("Date")
xlims!(1540, 1645)
xticks!(1540:30:1640, string.(Date.(julian2datetime.(t0 .+ collect(1540:30:1640)))))
ylims!(-0.39,0.6)
#savefig("doppler_shift_residuals__.pdf")

In [ ]:
plot(p2a, p2b, size=(640,860), layout=@layout([a; b]))
#savefig("doppler_shift_residuals__.pdf")
#savefig("~/phdthesis/fig5-7.pdf")

In [ ]:
plot(p1a, p1b, p2a, p2b, size=(700,600), layout=@layout([a b; c d]))
#savefig("/Users/Jorge/phdthesis/fig5-9.pdf")
#savefig("deldop_postfit_res_ATm9.pdf")

## Keeping initial conditions constant; varying only $A_2$:

In [ ]:
#x_myplot = range(-7e-14, -5e-14, length=100)
x_myplot = range(-15.65, 5.55, length=100)
Q_A2_deldop_1arg(x) = chi2(res_deldop, W_deldop)([0,0,0,0,0,0,x])
Q_A2_deldop_1arg.(x_myplot);

In [ ]:
plot(x_myplot, Q_A2_deldop_1arg.(x_myplot))

In [ ]:
qt1 = Q_A2_deldop_1arg(Taylor1(6))

In [ ]:
function taylor_minimum(pol::Taylor1{T}, x0::T; niters::Int=10) where {T<:Real}
    dpol = differentiate(pol)
    dpol2 = differentiate(dpol)
    xnewton::T = x0
    @show xnewton
    for i in 1:niters
        xnewton -= dpol(xnewton)/dpol2(xnewton) 
        @show xnewton, dpol(xnewton)
    end
    return xnewton
end

In [ ]:
taylor_minimum(qt1, 0.0)

## Plot above shows that, from radar data alone, it is not possible to obtain an estimation of $A_2$.

# Differential corrections, following Milani's approach

In [ ]:
#construct "design" matrix B
B_del = Matrix{TaylorN{Float64}}(undef, length(res_del), nv)
for i in 1:length(res_del)
    B_del[i,:] .= TaylorSeries.gradient(res_del[i])
end
B_dop = Matrix{TaylorN{Float64}}(undef, length(res_dop), nv)
for i in 1:length(res_dop)
    B_dop[i,:] .= TaylorSeries.gradient(res_dop[i])
end
B_deldop = Matrix{TaylorN{Float64}}(undef, length(res_deldop), nv)
for i in 1:length(res_deldop)
    B_deldop[i,:] .= TaylorSeries.gradient(res_deldop[i])
end

In [ ]:
# compute H = ∂²ξ/∂x²
H_del = Array{TaylorN{Float64}}(undef, length(res_del), nv, nv)
for i in 1:length(res_del)
    for j in 1:nv
            H_del[i,j,:] .= TaylorSeries.gradient(B_del[i,j])
    end
end
H_dop = Array{TaylorN{Float64}}(undef, length(res_dop), nv, nv)
for i in 1:length(res_dop)
    for j in 1:nv
            H_dop[i,j,:] .= TaylorSeries.gradient(B_dop[i,j])
    end
end
H_deldop = Array{TaylorN{Float64}}(undef, length(res_deldop), nv, nv)
for i in 1:length(res_deldop)
    for j in 1:nv
            H_deldop[i,j,:] .= TaylorSeries.gradient(B_deldop[i,j])
    end
end

In [ ]:
# compute transpose(ξ)*H
function ξTH(weights, residuals, Hmatrix, x)
    nv = get_numvars()
    ξTHv = Array{Float64}(undef, nv, nv)
    for j in 1:nv
        for i in 1:nv
           ξTHv[i,j] = (weights .* residuals(x))' * (Hmatrix[:,i,j](x))
        end
    end
    return ξTHv
enda

In [ ]:
function g(weights, residuals, Bmatrix, Hmatrix, x)
    return ((sqrt.(weights).*Bmatrix(x))' * (sqrt.(weights).*Bmatrix(x))) # + ξTH(weights, residuals, Hmatrix, x)
end

In [ ]:
inv(g(W_deldop, res_deldop, B_deldop, H_deldop, zeros(nv)))

In [ ]:
function f(weights, residuals, Bmatrix, x)
    return Bmatrix(x)' * (weights .* residuals(x))
end

In [ ]:
newton_LS_1iter(weights, residuals, Bmatrix, Hmatrix, x) = 
    inv(g(weights, residuals, Bmatrix, Hmatrix, x)) * f(weights, residuals, Bmatrix, x)

In [ ]:
function differential_corrections(weights, residuals, Bmatrix, Hmatrix, x0, niters::Int=40)
    x_new = x0
    for i in 1:niters
        Δx = -newton_LS_1iter(weights, residuals, Bmatrix, Hmatrix, x_new)
        x_new = x_new + Δx
        C_x0n = Bmatrix(x_new)' * (weights .* Bmatrix(x_new))
        Δx_C_Δx = (Δx') * C_x0n * Δx
        @show i, sqrt(Δx_C_Δx/length(Δx_C_Δx))
    end
    return x_new
end

In [ ]:
x0_JPL

In [ ]:
dx = zeros(7) #vcat(1e-7ones(6), 1.0).*randn(7)

In [ ]:
x_star_del = differential_corrections(W_del, res_del, B_del, H_del, x0_JPL+dx, 10)
x_star_dop = differential_corrections(W_dop, res_dop, B_dop, H_dop, x0_JPL+dx, 10)
x_star_deldop = differential_corrections(W_deldop, res_deldop, B_deldop, H_deldop, x0_JPL+dx, 10);

In [ ]:
# Apophis.jl + PlanetaryEphemeris.jl, ord=5
x_star_del

In [ ]:
# Apophis.jl + PlanetaryEphemeris.jl, ord=6
#x_star_del

In [ ]:
# Apophis.jl + PlanetaryEphemeris.jl, ord=7
#x_star_del

In [ ]:
# Apophis.jl + PlanetaryEphemeris.jl, ord=5
x_star_dop

In [ ]:
# Apophis.jl + PlanetaryEphemeris.jl, ord=6
#x_star_dop

In [ ]:
# Apophis.jl + PlanetaryEphemeris.jl, ord=7
#x_star_dop

In [ ]:
# Apophis.jl + PlanetaryEphemeris.jl, ord=5
x_star_deldop

In [ ]:
# Apophis.jl + PlanetaryEphemeris.jl, ord=6
#x_star_deldop

In [ ]:
# Apophis.jl + PlanetaryEphemeris.jl, ord=7
#x_star_deldop

In [ ]:
x0_JPL

Post-fit change in Initial conditions and $A_2$. Positions in $\mathrm{au}$, velocities in $\mathrm{au/day}$, Yarkovsky coefficient in $\mathrm{au/day}^2$

In [ ]:
using LinearAlgebra: norm

In [ ]:
x_star_deldop .* vcat( 1e-8ones(6), 1e-14)[1:nv] # delay + Doppler

In [ ]:
norm(ans[1:3]*au)

In [ ]:
norm(ans[4:6]*au/86400)

Post-fit change in Initial conditions and $A_2$. Positions in $\mathrm{km}$, velocities in $\mathrm{km/s}$, Yarkovsky coefficient in $\mathrm{au/day}^2$

In [ ]:
x_star_deldop .* vcat( 1e-8au*ones(3), 1e-8au*ones(3)/86400, 1e-14)[1:nv] # delay + Doppler

The value of the target function at the minimizer:

In [ ]:
rms(res_del(x_star_del), W_del)

In [ ]:
rms(res_dop(x_star_dop), W_dop)

In [ ]:
rms(res_deldop(x_star_deldop), W_deldop)

In [ ]:
rms(res_deldop(x_star_deldop+1e-6randn(7)), W_deldop)

In [ ]:
rms(res_deldop(x0_JPL), W_deldop)

The normal matrix evaluated at the minimizer:

w = 1/\sigma^2

In [ ]:
#B_deldop[1]

In [ ]:
B_deldop()

In [ ]:
C_del = B_del(x_star_del)' * (W_del .* B_del(x_star_del))

In [ ]:
C_dop = B_dop(x_star_dop)' * (W_dop .* B_dop(x_star_dop))

In [ ]:
C_deldop = B_deldop(x_star_deldop)' * (W_deldop .* B_deldop(x_star_deldop))

The inverse of the normal matrix $C$ is the covariance matrix $\Gamma$:

In [ ]:
Γ_del = inv(C_del)

In [ ]:
Γ_dop = inv(C_dop)

In [ ]:
Γ_deldop = inv(C_deldop)

Check that $\Gamma \cdot C \approx I_{7\times7}$

In [ ]:
Γ_deldop*C_deldop

The diagonal terms of the covariance matrix represent the 1-$\sigma$ uncertainty, squared, of each element of the minimizer; i.e., $\sigma_{ii} = \sqrt{\Gamma_{ii}}$

In [ ]:
Apophis.initialcond()+x_star_del .* vcat( 1e-8ones(6), 1e-14)[1:nv]

In [ ]:
sqrt.(diag(Γ_del)) .* vcat( 1e-8ones(6), 1e-14)[1:nv] # delay + Doppler

In [ ]:
x_star_deldop

In [ ]:
Γ_deldop

In [ ]:
d2Q_deldop_jt = TaylorSeries.hessian(Q_deldop, x_ts)

In [ ]:
d2Q_deldop_milani = (2/length(res_deldop))*g(W_deldop, res_deldop, B_deldop, H_deldop, x_ts)

In [ ]:
d2Q_deldop_milani = (2/length(res_deldop))*g(W_deldop, res_deldop, B_deldop, H_deldop, x_ts)

In [ ]:
# abs diff
norm(d2Q_deldop_jt-d2Q_deldop_milani, Inf)

In [ ]:
#rel diff
norm(d2Q_deldop_jt-d2Q_deldop_milani, Inf)/norm(d2Q_deldop_milani, -Inf)

In [ ]:
# abs diff
norm(d2Q_deldop_jt-d2Q_deldop_milani, Inf)

In [ ]:
#rel diff
norm(d2Q_deldop_jt-d2Q_deldop_milani, Inf)/norm(d2Q_deldop_milani, -Inf)

In [ ]:
# abs diff
norm(inv(d2Q_deldop_jt)-inv(d2Q_deldop_milani), Inf)

In [ ]:
#rel diff
norm(inv(d2Q_deldop_jt)-inv(d2Q_deldop_milani), Inf)/norm(inv(d2Q_deldop_milani), -Inf)

In [ ]:
Apophis.initialcond()

In [ ]:
sqrt.(diag(inv(d2Q_deldop_jt/(2/length(res_deldop))))) .* vcat(1e-8ones(6), 1e-14)

In [ ]:
sqrt.(diag(inv(d2Q_deldop_milani/(2/length(res_deldop))))) .* vcat(1e-8ones(6), 1e-14)